In [3]:
import pandas as pd     
import numpy as np 
import plotly.graph_objects as go

In [4]:
data = pd.read_csv("dataset_PCG.txt", sep='\t', header=None)

# Extract time (x-axis) and signal (y-axis)
x = data[0]
y = data[1]

# Create a plotly scatter plot
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=y, mode='lines', name='Signal'))

# Customize the layout
fig.update_layout(
    title="Signal vs Time",
    xaxis_title="Time (s)",
    yaxis_title="Signal",
    template="plotly"
)

In [5]:
coloumncount = 100
rowcount = 100
a = 0.0001
da = 0.0001
dt = 1/8000
f0 = 0.849
w0 = 2*np.pi* f0
Ndata = len(y)
pi = np.pi

db = (Ndata - 1) * dt / coloumncount
cwtre = np.zeros((coloumncount, rowcount))
cwtim = np.zeros((coloumncount, rowcount))
cwt = np.zeros((coloumncount, rowcount))

db = (Ndata-1)* dt / coloumncount
for i in range(rowcount):
    b = 0.0
    for j in range(coloumncount):
        t = 0.0
        cwtre_sum = 0.0
        cwtim_sum = 0.0
        for k in range(Ndata):
            rem = (1 / np.sqrt(a)) * (1 / np.power(pi, 0.25)) * np.exp(-((t - b) / a) ** 2 / 2.0) * np.cos(w0 * (t - b) / a)
            imm = (1 / np.sqrt(a)) * (-1 / np.power(pi, 0.25)) * np.exp(-((t - b) / a) ** 2 / 2.0) * np.sin(w0 * (t - b) / a)
            cwtre_sum += y[k] * rem
            cwtim_sum += y[k] * imm
            t += dt
        
        cwtre[j, i] = cwtre_sum
        cwtim[j, i] = cwtim_sum
        cwt[j, i] = np.sqrt(cwtre[j, i] ** 2 + cwtim[j, i] ** 2)
        b += db
    
    a += da

In [6]:
X, Y = np.meshgrid(np.arange(rowcount), np.arange(coloumncount))
Z = cwt.T  

fig = go.Figure(data=[go.Surface(z=Z, x=X, y=Y, colorscale='Viridis')])
fig.update_layout(title='Continuous Wavelet Transform (CWT) 3D Plot',
                  scene=dict(xaxis_title='Scale',
                             yaxis_title='Freq',
                             zaxis_title='Magnitude'),
                  autosize=True)
fig.show()

In [7]:
x = np.arange(coloumncount+ 1)
y = np.arange(rowcount + 1)

# Create 2D Contour Plot
fig_2d = go.Figure(
    data=go.Contour(
        z=cwt,  # 2D array for z values
        x=x,  # x-axis values (translation)
        y=y,  # y-axis values (scale)
        contours_coloring="heatmap",
        line_smoothing=0.85,
        colorbar=dict(title="Magnitude"),
    )
)

fig_2d.update_layout(
    title="2D Contour Plot", xaxis_title="Translation", yaxis_title="Scale", height=500, width=500
)

In [8]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Data untuk plot 3D
X, Y = np.meshgrid(np.arange(rowcount), np.arange(coloumncount))
Z = cwt.T  

# Membuat subplot dengan 2 kolom
fig = make_subplots(
    rows=1, cols=2,  # 1 baris, 2 kolom
    subplot_titles=("3D Surface Plot", "2D Contour Plot"),
    specs=[[{'type': 'surface'}, {'type': 'contour'}]],  # Tipe untuk masing-masing subplot
    column_widths=[0.5, 0.5]  # Lebar relatif kolom
)

# Plot 3D Surface
fig.add_trace(
    go.Surface(
        z=Z, 
        x=X, 
        y=Y, 
        colorscale='Jet',
        contours_z=dict(
            show=True,  # Menampilkan kontur di sumbu Z
            usecolormap=True,  # Menggunakan skala warna
            highlightcolor="Forestgreen",  # Warna highlight
            project_z=True  # Proyeksikan kontur ke sumbu Z
        )
    ),
    row=1, col=1  # Ditempatkan di baris 1 kolom 1
)

# Data untuk plot 2D Contour
y = np.arange(coloumncount + 1)
x = np.arange(rowcount + 1)

# Plot 2D Contour
fig.add_trace(
    go.Contour(
        z=cwt.T,
        y=y,
        x=x,
        colorscale='Jet',
        contours_coloring="heatmap",
        line_smoothing=0.85,
        colorbar=dict(title="Magnitude"),
    ),
    row=1, col=2  # Ditempatkan di baris 1 kolom 2
)

# Memperbarui layout keseluruhan

fig.update_layout(
    title_text="3D Surface and 2D Contour Plot", 
    height=600,  # Tinggi figure
    width=1000,  # Lebar figure
    scene=dict(
        xaxis_title='Scale',  # Label sumbu x untuk 3D plot
        yaxis_title='Freq',   # Label sumbu y untuk 3D plot
        zaxis_title='Magnitude'  # Label sumbu z untuk 3D plot
    ),
    autosize=True,
    xaxis2=dict(title="X Axis Label"),  # Label sumbu x untuk 2D Contour
    yaxis2=dict(title="Y Axis Label")   # Label sumbu y untuk 2D Contour
)

# Menampilkan plot
fig.show()


In [19]:
import pandas as pd
import numpy as np

# Parameters for scaling
coloumncount = 100
rowcount = 100
a_awal = 0.0001  # initial scale
da = 0.0009  # delta scale
f0 = 0.849  # center frequency

# Max time for the dataset
time_max = np.max(data[0])

# Initialize arrays for frequency and time scaling
fk = np.zeros(rowcount)  # Frequency scale
tk = np.zeros(coloumncount)  # Time scale

# Calculate frequency scale fk[i]
for i in range(rowcount):
    fk[i] = f0 / (a_awal + (i * da))

# Calculate time scale tk[j]
for j in range(coloumncount):
    tk[j] = (time_max / coloumncount) * j

# Create a DataFrame to store the scales
df_scale_conversion = pd.DataFrame({
    'Scale_Index': np.arange(0, rowcount ),
    'Frequency_Hz': fk,
    'Time_ms': tk[:rowcount]  # Adjusting size to rowcount
})

# Display the DataFrame
print(df_scale_conversion)


    Scale_Index  Frequency_Hz  Time_ms
0             0   8490.000000  0.00000
1             1    849.000000  0.01999
2             2    446.842105  0.03998
3             3    303.214286  0.05997
4             4    229.459459  0.07996
..          ...           ...      ...
95           95      9.918224  1.89905
96           96      9.815029  1.91904
97           97      9.713959  1.93903
98           98      9.614949  1.95902
99           99      9.517937  1.97901

[100 rows x 3 columns]
